In [1]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
train = pd.read_csv("./data/raw_data/train.csv", parse_dates=['date'])
test = pd.read_csv('./data/raw_data/test.csv', parse_dates=['date'])

## EDA

### Target

In [3]:
fig = go.Figure()

for warehouse in train['warehouse'].unique():
    fig.add_trace(
        go.Scatter(
            x=train.loc[train['warehouse'] == warehouse, 'date'],
            y=train.loc[train['warehouse'] == warehouse, 'orders'],
            mode='lines',
            name=warehouse
        )
    )

fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    xaxis_title="Date",
    yaxis_title="Orders",
)

fig.show()

In [4]:
warehouses = sorted(train['warehouse'].unique())

fig = make_subplots(
    rows=train['warehouse'].nunique(),
    cols=3,
    column_widths=[0.6, 0.2, 0.2],
    row_titles=warehouses
)

for row, warehouse in enumerate(warehouses):
    sub_data = train.loc[train['warehouse'] == warehouse]
    
    fig.add_trace(
        go.Scatter(
            x=sub_data['date'],
            y=sub_data['orders'],
            mode='lines',
            name=warehouse,
            showlegend=False
        ),
        row=row+1,
        col=1
    )
    
    fig.add_trace(
        go.Histogram(
            x=sub_data['orders'],
            name=warehouse,
            showlegend=False
        ),
        row=row+1,
        col=2
    )
    
    fig.add_trace(
        go.Box(
            y=sub_data['orders'],
            # boxpoints='all',
            # jitter=0.3,
            # pointpos=-1.8,
            name=warehouse,
            showlegend=False
        ),
        row=row+1,
        col=3
    )
    
fig.update_layout(
    autosize=False,
    width=1100,
    height=2000,
)    

fig.for_each_annotation(lambda a:   a.update(x = -0.07) if a.text in warehouses else())

fig.show()

In [5]:
train["year_month"] = train["date"].dt.to_period("M")

grouped_mean = train.groupby(["warehouse", "year_month"])["orders"].mean()

grouped_mean = grouped_mean.reset_index().rename(columns={"orders": "mean_orders"})

train = train.merge(grouped_mean, on=["warehouse", "year_month"], how="left")


fig = go.Figure()

for warehouse in train["warehouse"].unique():
    fig.add_trace(
        go.Scatter(
            x=train.loc[train["warehouse"] == warehouse, "date"],
            y=train.loc[train["warehouse"] == warehouse, "mean_orders"],
            mode="lines",
            name=warehouse,
        )
    )

fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    xaxis_title="Date",
    yaxis_title="Orders",
)

fig.show()

### Drivers

In [6]:
modelling_cols = list(test.columns) + ['orders']

train[modelling_cols]

,warehouse,date,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,id,orders
0,Prague_1,2020-12-05,NaN,0,0,0,0,Prague_1_2020-12-05,6895.0
1,Prague_1,2020-12-06,NaN,0,0,0,0,Prague_1_2020-12-06,6584.0
2,Prague_1,2020-12-07,NaN,0,0,0,0,Prague_1_2020-12-07,7030.0
3,Prague_1,2020-12-08,NaN,0,0,0,0,Prague_1_2020-12-08,6550.0
4,Prague_1,2020-12-09,NaN,0,0,0,0,Prague_1_2020-12-09,6910.0
...,...,...,...,...,...,...,...,...,...
7335,Budapest_1,2024-03-10,NaN,0,0,0,0,Budapest_1_2024-03-10,6733.0
7336,Budapest_1,2024-03-11,NaN,0,0,0,0,Budapest_1_2024-03-11,6492.0
7337,Budapest_1,2024-03-12,NaN,0,0,0,0,Budapest_1_2024-03-12,6661.0
7338,Budapest_1,2024-03-13,NaN,0,0,0,0,Budapest_1_2024-03-13,6843.0


#### Holidays 

In [7]:
train['next_holiday_date'] = pd.NaT

train.loc[train["holiday"] == 1, "next_holiday_date"] = train.loc[
    train["holiday"] == 1, "date"
]
train['next_holiday_date'] = train.groupby("warehouse")["next_holiday_date"].bfill()


train["days_until_next_holiday"] = train["next_holiday_date"] - train["date"]

train

,warehouse,date,orders,holiday_name,holiday,shutdown,mini_shutdown,shops_closed,winter_school_holidays,school_holidays,...,frankfurt_shutdown,precipitation,snow,user_activity_1,user_activity_2,id,year_month,mean_orders,next_holiday_date,days_until_next_holiday
0,Prague_1,2020-12-05,6895.0,NaN,0,0,0,0,0,0,...,0,0.00,0.0,1722.0,32575.0,Prague_1_2020-12-05,2020-12,7315.692308,2020-12-24,19 days
1,Prague_1,2020-12-06,6584.0,NaN,0,0,0,0,0,0,...,0,0.00,0.0,1688.0,32507.0,Prague_1_2020-12-06,2020-12,7315.692308,2020-12-24,18 days
2,Prague_1,2020-12-07,7030.0,NaN,0,0,0,0,0,0,...,0,0.00,0.0,1696.0,32552.0,Prague_1_2020-12-07,2020-12,7315.692308,2020-12-24,17 days
3,Prague_1,2020-12-08,6550.0,NaN,0,0,0,0,0,0,...,0,0.80,0.0,1681.0,32423.0,Prague_1_2020-12-08,2020-12,7315.692308,2020-12-24,16 days
4,Prague_1,2020-12-09,6910.0,NaN,0,0,0,0,0,0,...,0,0.50,0.0,1704.0,32410.0,Prague_1_2020-12-09,2020-12,7315.692308,2020-12-24,15 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7335,Budapest_1,2024-03-10,6733.0,NaN,0,0,0,0,0,0,...,0,1.51,0.0,2983.0,27111.0,Budapest_1_2024-03-10,2024-03,6869.142857,NaT,NaT
7336,Budapest_1,2024-03-11,6492.0,NaN,0,0,0,0,0,0,...,0,1.03,0.0,2975.0,27133.0,Budapest_1_2024-03-11,2024-03,6869.142857,NaT,NaT
7337,Budapest_1,2024-03-12,6661.0,NaN,0,0,0,0,0,0,...,0,0.21,0.0,2974.0,27151.0,Budapest_1_2024-03-12,2024-03,6869.142857,NaT,NaT
7338,Budapest_1,2024-03-13,6843.0,NaN,0,0,0,0,0,0,...,0,0.00,0.0,2979.0,27180.0,Budapest_1_2024-03-13,2024-03,6869.142857,NaT,NaT


In [8]:
avg_orders_per_day_until_holiday = pd.crosstab(
    train["days_until_next_holiday"],
    train["warehouse"],
    values=train["orders"],
    aggfunc="mean",
)

fig = go.Figure()

for warehouse, values in avg_orders_per_day_until_holiday.items():
    fig.add_trace(
        go.Scatter(
            y=values,
            mode="lines",
            name=warehouse,
        )
    )

fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    xaxis_title="Number of days until an holiday",
    yaxis_title="Average order",
)

fig.show()

##### Orders per number of days until each holiday for all warehouse 

In [9]:
train["next_holiday_name"] = None

train.loc[train["holiday"] == 1, "next_holiday_name"] = train.loc[
    train["holiday"] == 1, "holiday_name"
]

train.loc[
    (train["holiday"] == 1) & (train["next_holiday_name"].isna()), "next_holiday_name"
] = "MISSING_NAME"
train["next_holiday_name"] = train.groupby("warehouse")["next_holiday_name"].bfill()

train

,warehouse,date,orders,holiday_name,holiday,shutdown,mini_shutdown,shops_closed,winter_school_holidays,school_holidays,...,precipitation,snow,user_activity_1,user_activity_2,id,year_month,mean_orders,next_holiday_date,days_until_next_holiday,next_holiday_name
0,Prague_1,2020-12-05,6895.0,NaN,0,0,0,0,0,0,...,0.00,0.0,1722.0,32575.0,Prague_1_2020-12-05,2020-12,7315.692308,2020-12-24,19 days,Christmas Eve
1,Prague_1,2020-12-06,6584.0,NaN,0,0,0,0,0,0,...,0.00,0.0,1688.0,32507.0,Prague_1_2020-12-06,2020-12,7315.692308,2020-12-24,18 days,Christmas Eve
2,Prague_1,2020-12-07,7030.0,NaN,0,0,0,0,0,0,...,0.00,0.0,1696.0,32552.0,Prague_1_2020-12-07,2020-12,7315.692308,2020-12-24,17 days,Christmas Eve
3,Prague_1,2020-12-08,6550.0,NaN,0,0,0,0,0,0,...,0.80,0.0,1681.0,32423.0,Prague_1_2020-12-08,2020-12,7315.692308,2020-12-24,16 days,Christmas Eve
4,Prague_1,2020-12-09,6910.0,NaN,0,0,0,0,0,0,...,0.50,0.0,1704.0,32410.0,Prague_1_2020-12-09,2020-12,7315.692308,2020-12-24,15 days,Christmas Eve
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7335,Budapest_1,2024-03-10,6733.0,NaN,0,0,0,0,0,0,...,1.51,0.0,2983.0,27111.0,Budapest_1_2024-03-10,2024-03,6869.142857,NaT,NaT,NaN
7336,Budapest_1,2024-03-11,6492.0,NaN,0,0,0,0,0,0,...,1.03,0.0,2975.0,27133.0,Budapest_1_2024-03-11,2024-03,6869.142857,NaT,NaT,NaN
7337,Budapest_1,2024-03-12,6661.0,NaN,0,0,0,0,0,0,...,0.21,0.0,2974.0,27151.0,Budapest_1_2024-03-12,2024-03,6869.142857,NaT,NaT,NaN
7338,Budapest_1,2024-03-13,6843.0,NaN,0,0,0,0,0,0,...,0.00,0.0,2979.0,27180.0,Budapest_1_2024-03-13,2024-03,6869.142857,NaT,NaT,NaN


In [10]:
warehouses = sorted(train["warehouse"].unique())

fig = make_subplots(
    rows=train["warehouse"].nunique(),
    cols=1,
    row_titles=warehouses,
)

for row, warehouse in enumerate(warehouses):
    sub_data = train.loc[train["warehouse"] == warehouse]

    avg_orders = pd.crosstab(
        sub_data["days_until_next_holiday"],
        sub_data["next_holiday_name"],
        values=sub_data["orders"],
        aggfunc="mean",
    )

    for holiday, values in avg_orders.items():
        fig.add_trace(
            go.Scatter(y=values, mode="lines", name=holiday, showlegend=False),
            row=row + 1,
            col=1,
        )

fig.update_layout(
    autosize=False,
    width=1100,
    height=2000,
    # legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
)

fig.for_each_annotation(lambda a: a.update(x=-0.07) if a.text in warehouses else ())

fig.show()

#### Orders per day of week

There are more orders on Friday for every location.

Frankfurt_1 does not work on sunday, hence saturday and monday have a bit more than the rest to compensate.

In [11]:
train["year"] = train["date"].dt.year
train["month"] = train["date"].dt.month
train["day"] = train["date"].dt.day
train["week"] = train["date"].dt.isocalendar().week
train["dayofweek"] = train["date"].dt.dayofweek
train["is_weekend"] = train["dayofweek"].isin([5, 6]).astype(int)

In [12]:
days_of_week_legend = {
    0:'Monday',
    1:'Thuesday',
    2:'Wednesday',
    3:'Thursday',
    4:'Friday',
    5:'Saturday',
    6:'Sunday'
}


avg_orders_per_day_of_week = pd.crosstab(
    train["dayofweek"],
    train["warehouse"],
    values=train["orders"],
    aggfunc="mean",
)

avg_orders_per_day_of_week = avg_orders_per_day_of_week.rename(index=days_of_week_legend)

avg_orders_per_day_of_week

warehouse,Brno_1,Budapest_1,Frankfurt_1,Munich_1,Prague_1,Prague_2,Prague_3
dayofweek,,,,,,,
Monday,7051.488235,5469.025316,1524.755102,3594.403226,8562.776471,5033.800000,4624.317647
Thuesday,7101.011696,5534.672619,1408.932692,3071.705426,8304.877193,4984.017544,4563.865497
Wednesday,7146.824561,5494.088757,1393.768519,2983.458647,8326.000000,4995.210526,4587.871345
Thursday,7304.754386,5584.730994,1422.269231,3058.438462,8537.847953,5126.771930,4705.350877
Friday,8084.923529,5977.817073,1585.700935,3780.671642,9395.417647,5722.847059,5218.982353
Saturday,7320.158824,5583.812121,1655.981481,3931.189394,8319.870588,5081.317647,4663.641176
Sunday,6914.158824,5234.943396,NaN,1351.000000,8303.141176,5052.570588,4578.882353


In [13]:
fig = go.Figure()

for day_of_week, values in avg_orders_per_day_of_week.T.items():
    fig.add_trace(
        go.Bar(
            y=values,
            x=values.index,
            name=day_of_week,
        )
    )

fig.update_layout(
    # legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    xaxis_title="Warehouse",
    yaxis_title="Orders",
)

fig.show()